<a href="https://colab.research.google.com/github/kgeneral/Hands-On-MachineLearning-2nd/blob/master/19_training_and_deploying_at_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chapter 19 – Training and Deploying TensorFlow Models at Scale**

In [ ]:
# 기본 설정

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
    !apt update && apt-get install -y tensorflow-model-server
    !pip install -q -U tensorflow-serving-api
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deploy"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   191k      0 --:--:-- --:--:-- --:--:--  191k
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.laun

In [ ]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/lib/python3.6/dist-packages (1.10.0)


# 텐서플로 모델 서빙

## Tensorflow Serving 

- TF 모델의 추론결과값을 효율적으로 서비스화(서빙) 하기 위한 프레임워크
- 자체 하드웨어 또는 GCP AI 등에서 사용 가능
- REST API 또는 gRPC API 사용 사능
- **SavedModel 포맷으로 저장된 모델을 사용함**

![TFS](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00175.jpg)

### Save/Load a `SavedModel`

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7012 - accuracy: 0.8241 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3536 - accuracy: 0.9020 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3036 - accuracy: 0.9145 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2736 - accuracy: 0.9231 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2509 - accuracy: 0.9296 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2322 - accuracy: 0.9350 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2161 - accuracy: 0.9401 - val_loss: 0.1970 - val_accuracy:

In [ ]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [ ]:
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model/0001'

In [ ]:
!rm -rf {model_name}

In [ ]:
tf.saved_model.save(model, model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


### SavedModel 구조

- saved_model.pb : 모델의 computation graph
- variables : 모델의 variable
- assets : 모델에 필요한 임의 파일(어휘 사전 등) 을 넣을 수 있음
  - https://www.tensorflow.org/api_docs/python/tf/saved_model/Asset

In [ ]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_mnist_model/
    0001/
        saved_model.pb
        variables/
            variables.data-00001-of-00002
            variables.index
            variables.data-00000-of-00002
        assets/


In [ ]:
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
serve


In [ ]:
!saved_model_cli show --dir {model_path} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [ ]:
!saved_model_cli show --dir {model_path} --tag_set serve \
                      --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [ ]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0802 07:26:20.064532 140579382863744 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/p

Let's write the new instances to a `npy` file so we can pass them easily to our model:

### numpy 파일의 cli inference

- saved_model_cli 에 npy 파일을 입력하여 inference 할 수 있다. (배치성 데이터의 프로세싱 등)

In [ ]:
# inference 할 feature data를 저장
np.save("my_mnist_tests.npy", X_new)

In [ ]:
input_name = model.input_names[0]
input_name

'flatten_input'

And now let's use `saved_model_cli` to make predictions for the instances we just saved:

In [ ]:
!saved_model_cli run --dir {model_path} --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=my_mnist_tests.npy

2020-08-02 07:26:23.364521: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-02 07:26:23.366973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-02 07:26:23.367883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-08-02 07:26:23.368154: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-02 07:26:23.370201: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-08-02 07:26:23.372098: I tensorflow/stream_executor/plat

In [ ]:
np.round([[1.1739199e-04, 1.1239604e-07, 6.0210604e-04, 2.0804715e-03, 2.5779348e-06,
           6.4079795e-05, 2.7411186e-08, 9.9669880e-01, 3.9654213e-05, 3.9471846e-04],
          [1.2294615e-03, 2.9207937e-05, 9.8599273e-01, 9.6755642e-03, 8.8930705e-08,
           2.9156188e-04, 1.5831805e-03, 1.1311053e-09, 1.1980456e-03, 1.1113169e-07],
          [6.4066830e-05, 9.6359509e-01, 9.0598064e-03, 2.9872139e-03, 5.9552520e-04,
           3.7478798e-03, 2.5074568e-03, 1.1462728e-02, 5.5553433e-03, 4.2495009e-04]], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.96, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.01, 0.  ]])

## TensorFlow Serving with Docker

### docker 를 이용하는 설치 방법
- https://docs.docker.com/get-started/part2/
    - tensorflow/serving 이미지를 docker hub 에서 다운로드 받아서 
    - $ML_PATH/my_mnist_model 로컬 위치의 모델을 docker 이미지 내부의 /models/my_mnist_model 경로로 바인딩하고
    - docker 이미지 os 내부에 MODEL_NAME=my_mnist_model 환경 변수를 지정하고
    - 8501 포트를 사용해 로컬 머신 에서 띄우는 커맨드
```bash
docker pull tensorflow/serving
export ML_PATH=$HOME/ml # or wherever this project is
docker run -it --rm -p 8500:8500 -p 8501:8501 \
   -v "$ML_PATH/my_mnist_model:/models/my_mnist_model" \
   -e MODEL_NAME=my_mnist_model \
   tensorflow/serving
```
Once you are finished using it, press Ctrl-C to shut down the server.

colab 환경 에서는 `tensorflow_model_server` 를 커스텀 설치 하여 사용

In [ ]:
os.environ["MODEL_DIR"] = os.path.split(os.path.abspath(model_path))[0]

In [ ]:
%%bash --bg
nohup tensorflow_model_server \
     --rest_api_port=8501 \
     --model_name=my_mnist_model \
     --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [ ]:
!tail server.log

2020-08-02 07:26:25.557338: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:183] Running initialization op on SavedModel bundle at path: /content/my_mnist_model/0001
2020-08-02 07:26:25.562282: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:364] SavedModel load for tags { serve }; Status: success: OK. Took 37666 microseconds.
2020-08-02 07:26:25.562816: I tensorflow_serving/servables/tensorflow/saved_model_warmup.cc:105] No warmup data file found at /content/my_mnist_model/0001/assets.extra/tf_serving_warmup_requests
2020-08-02 07:26:25.562933: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: my_mnist_model version: 1}
2020-08-02 07:26:25.564235: I tensorflow_serving/model_servers/server.cc:355] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] getaddrinfo: address family for nodename not supported
[evhttp_server.cc : 223] NET_LOG: Couldn't bind to port 8501
[evhttp_server.cc : 63] NET_LOG: Server has not been te

In [ ]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [ ]:
repr(input_data_json)[:1500] + "..."

'\'{"signature_name": "serving_default", "instances": [[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0

### REST API 를 이용한 호출

In [ ]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [ ]:
response.keys()

dict_keys(['predictions'])

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### gRPC API 를 이용한 호출

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [ ]:
response

outputs {
  key: "dense_1"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 3
      }
      dim {
        size: 10
      }
    }
    float_val: 0.00011425150296417996
    float_val: 1.5136735953547031e-07
    float_val: 0.0009818377438932657
    float_val: 0.0027773668989539146
    float_val: 3.7589061321341433e-06
    float_val: 7.62673225835897e-05
    float_val: 3.9139663954301795e-08
    float_val: 0.995561957359314
    float_val: 5.344559394870885e-05
    float_val: 0.000430887594120577
    float_val: 0.0008195420377887785
    float_val: 3.5499859222909436e-05
    float_val: 0.9882416129112244
    float_val: 0.00705768121406436
    float_val: 1.2937896087805711e-07
    float_val: 0.00023403267550747842
    float_val: 0.002574468730017543
    float_val: 9.668648104366184e-10
    float_val: 0.0010369742522016168
    float_val: 8.834076936636848e-08
    float_val: 4.441462806425989e-05
    float_val: 0.970328688621521
    float_val: 0.009044435806572437
    

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### 새로운 model 의 배포 적용

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7039 - accuracy: 0.8056 - val_loss: 0.3418 - val_accuracy: 0.9042
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3204 - accuracy: 0.9082 - val_loss: 0.2674 - val_accuracy: 0.9242
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2650 - accuracy: 0.9235 - val_loss: 0.2227 - val_accuracy: 0.9368
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2319 - accuracy: 0.9329 - val_loss: 0.2032 - val_accuracy: 0.9432
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2089 - accuracy: 0.9399 - val_loss: 0.1833 - val_accuracy: 0.9482
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1908 - accuracy: 0.9446 - val_loss: 0.1740 - val_accuracy: 0.9498
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1756 - accuracy: 0.9490 - val_loss: 0.1605 - val_accuracy:

In [ ]:
model_version = "0002"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model/0002'

In [ ]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


- 신규 버젼의 모델(0002) 이 my_mnist_model 에 추가됨

In [ ]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_mnist_model/
    0001/
        saved_model.pb
        variables/
            variables.data-00001-of-00002
            variables.index
            variables.data-00000-of-00002
        assets/
    0002/
        saved_model.pb
        variables/
            variables.data-00001-of-00002
            variables.index
            variables.data-00000-of-00002
        assets/


In [ ]:
!zip -r my_mnist_model.zip my_mnist_model

updating: my_mnist_model/ (stored 0%)
updating: my_mnist_model/0001/ (stored 0%)
updating: my_mnist_model/0001/saved_model.pb (deflated 87%)
updating: my_mnist_model/0001/variables/ (stored 0%)
updating: my_mnist_model/0001/variables/variables.data-00001-of-00002 (deflated 8%)
updating: my_mnist_model/0001/variables/variables.index (deflated 52%)
updating: my_mnist_model/0001/variables/variables.data-00000-of-00002 (deflated 72%)
updating: my_mnist_model/0001/assets/ (stored 0%)
updating: my_mnist_model/0002/ (stored 0%)
updating: my_mnist_model/0002/saved_model.pb (deflated 88%)
updating: my_mnist_model/0002/variables/ (stored 0%)
updating: my_mnist_model/0002/variables/variables.data-00001-of-00002 (deflated 8%)
updating: my_mnist_model/0002/variables/variables.index (deflated 54%)
updating: my_mnist_model/0002/variables/variables.data-00000-of-00002 (deflated 75%)
updating: my_mnist_model/0002/assets/ (stored 0%)


In [ ]:
# colab 내의 파일 다운로드
from google.colab import files
files.download('my_mnist_model.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- 수 분 정도의 시간 이후 신규 모델이 적용됨


![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00238.jpg)

In [ ]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
            
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status()
response = response.json()

In [ ]:
response.keys()

dict_keys(['predictions'])

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

## GCP AI Platform 에서 예측 서비스 만들기

### 사전준비
- 실습해 보기 위해서는 GCP 가입 필요(휴대폰 인증 및 카드정보 등록 필요)
- 새 계정 하나 만들어서 free tier 쓰는 것 추천(요즘은 가입시 폰번호 입력 안 받는 듯함)

### GCP AI 플랫폼 quickstart
- 구글계정 로그인 및 google cloud console 접속 : https://console.cloud.google.com/
- 좌측메뉴 > Storage > Browser
- 모델 파일을 업로드할 GCS bucket 을 생성
   - us-central1, europe-west4, asia-east1 추천 : AI Platform 에서 지원되는 region
- 생성한 saved_model 을 GCS bucket 에 업로드(base path 기준으로)

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00183.jpg)

- 좌측메뉴 > AI Platform > Models
- 처음이면 Enable API 클릭
- Create Model
  - GCS Bucket 과 같은 region 선택 
- Create version(5분 이상 소요) : 프레임워크, 하드웨어 등 세부 설정이 들어감
  - Python version
  - Ml framework(scikit, TF, XGBoost 등)
  - Accelerator(GPU) 장비 종류 및 unit 단위
  - GCS 경로(browse 가능)
  - Autoscaling 여부 옵션


### 예측 서비스 사용하기
- 본 예제에서는 google api python sdk 를 사용

#### 서비스 전용 credential 생성
- Public cloud 활용 시 root credential 은 권한범위가 넓어서 위험하고 잘 사용하지 않음
- 따라서 권한이 제한된 서비스 계정을 생성하고 json credential 을 만들고 연동하여 사용하는 것이 보통

#### Service account 추가
- 좌측메뉴 > IAM & Admin > Service account > Create service account
- 생성된 계정 클릭 > Add keys

#### IAM
- 좌측메뉴 > IAM & Admin > IAM > Permission > ADD
- 위에서 생성한 서비스 계정 선택, Role 에 ML Engine Developer 추가 

In [ ]:
#!wget -O cred.json "https://drive.google.com/u/0/uc?id=1eDk-dUdOno3RjOsiE6WsJujuwJ8sUJKN&export=download"

--2020-08-02 07:27:19--  https://drive.google.com/u/0/uc?id=1eDk-dUdOno3RjOsiE6WsJujuwJ8sUJKN&export=download
Resolving drive.google.com (drive.google.com)... 172.217.203.100, 172.217.203.102, 172.217.203.113, ...
Connecting to drive.google.com (drive.google.com)|172.217.203.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fvp3epjlcskh1rf3ohvqjsdoegmbql4i/1596353175000/14905503537404824244/*/1eDk-dUdOno3RjOsiE6WsJujuwJ8sUJKN?e=download [following]
--2020-08-02 07:27:19--  https://doc-0s-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fvp3epjlcskh1rf3ohvqjsdoegmbql4i/1596353175000/14905503537404824244/*/1eDk-dUdOno3RjOsiE6WsJujuwJ8sUJKN?e=download
Resolving doc-0s-ak-docs.googleusercontent.com (doc-0s-ak-docs.googleusercontent.com)... 74.125.26.132, 2607:f8b0:400c:c04::84
Connecting to doc-0s-ak-docs.googleusercontent.com 

In [ ]:
# ml engine sdk 최근 샘플 코드 : Use your deployed model > Sample prediction request
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cred.json"

import googleapiclient.discovery
def predict_json(project, region, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        region (str): regional endpoint to use; set to None for ml.googleapis.com
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    prefix = "{}-ml".format(region) if region else "ml"
    api_endpoint = "https://{}.googleapis.com".format(prefix)
    client_options = dict(api_endpoint=api_endpoint)
    service = googleapiclient.discovery.build(
        'ml', 'v1', client_options=client_options)
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']


predict_json("profound-gantry-285109", "us-central1", "my_mnist_model", X_new.tolist(), version="v0001")

[[0.000113918402,
  1.51716876e-07,
  0.000975968898,
  0.00278414763,
  3.75585432e-06,
  7.64351134e-05,
  3.90252559e-08,
  0.995564222,
  5.29630342e-05,
  0.000428568979],
 [0.000814454,
  3.52761344e-05,
  0.988261163,
  0.00706663728,
  1.2855574e-07,
  0.000233507817,
  0.00255494262,
  9.62903202e-10,
  0.00103384326,
  8.75075301e-08],
 [4.43605459e-05,
  0.970256,
  0.00910014,
  0.00227546599,
  0.000487301499,
  0.00287919072,
  0.00227399077,
  0.00836221408,
  0.00402356684,
  0.000297729333]]

# 모바일 또는 임베드 장치에 모델 배포하기


## TF Lite
- https://www.tensorflow.org/lite/convert
- TF Lite converter 를 이용해여 모델을 축소
- 실제 모델 그래프의 최적화도 진행 

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter.optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
with open("converted_model.tflite","wb") as f:
  f.write(tflite_model)


In [ ]:
!ls -al converted_model.tflite

-rw-r--r-- 1 root root 171264 Aug  1 23:48 converted_model.tflite


## 모델 그래프 최적화의 정도 
- 큰 모델의 tflite, pb 비교해보면 알 수 있음 : https://www.tensorflow.org/lite/guide/hosted_models
- TF 모델 그래프 조회 툴 : https://lutzroeder.github.io/netron/


## Post-training quantization
- 모델의 weight 자료구조의 bit size 를 축소하는 기법
   - 32bit float > 8bit int
- abs(weight) 의 최대값을 기준으로 -127 ~ 127(signed 8bit int) scalar space 에 매핑

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00188.jpg)

## Quantization 된 모델의 사용
- float 로 복원하여 사용 : RAM사용량, 실행속도 등에서 이득이 없음
- activation 출력을 quantization 처리하여 전체 내부 연산을 정수화
  - Edge TPU 등

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter.optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
with open("converted_model_quantized.tflite","wb") as f:
  f.write(tflite_model)

In [ ]:
!ls -al converted_model_quantized.tflite

-rw-r--r-- 1 root root 46264 Aug  2 00:30 converted_model_quantized.tflite


# GPU 를 통한 계산 속도 향상
- TF GPU 지원 : https://www.tensorflow.org/install/gpu?hl=ko
   - Compute Capability >= 3.5
   - CUDA = 10.1
   - cuDNN SDK >= 7.6

![대체 텍스트](https://image.slidesharecdn.com/intototensorflowarchitecturev2-170703174758/95/an-introduction-to-tensorflow-architecture-7-638.jpg?cb=1520314886)

- GPU 활성화여부 및 메타 정보 반환 등의 커맨드

In [ ]:
tf.test.is_gpu_available() # 최소 1개 이상의 GPU 가 사용 가능하면 True

In [ ]:
tf.test.gpu_device_name()

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
!nvidia-smi

In [ ]:
from tensorflow.python.client.device_lib import list_local_devices

devices = list_local_devices()
devices

## Public cloud 의 GPU VM 이용하기
- GCP 의 경우 기본적으로는 GPU VM 할당량이 0 으로 되어 잇음.
  - 할당량 수정을 요청하여 승인을 받으면 승인받은 수만큼 GPU VM 을 사용할 수 잇음


## Colab 이용하기
- Standard : 무료
- Pro : $9.9 / month
  - High RAM 등의 추가 옵션 제공

## GPU RAM 관리
- 기본적으로 TF 는 실행시점에 가능한 모든 GPU 의 RAM 을 확보
  - Memory fragmentation 을 방지하기 위함



### Process 별 device직접 분배
- 환경 변수를 통해 BUS_ID 지정

```bash
CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICE=0,1 python3 program1.py
CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICE=3,2 python3 program2.py
```

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00201.jpg)


### TF Process 별 명시적 메모리 limit 값 지정

```python
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_virtual_device_configuration(
        gpu,
        [ tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
```

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00202.jpg)

### 필요할 때에 매모리를 점진적으로 점유하는 옵션

```python
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)
```


### 물리 GPU 를 여러 가상 GPU 로 지정하는 전략

```python
physical_gpus=tf.config.experimental.list_physcal_devices("GPU")
tf.config.experimental.set_virtual_device_configuration(
    physical_gpus[0],
    [ tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
      tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
```


## 디바이스에 연산과 변수 할당하기
- 보통 데이터 전처리를 CPU, 신경망 연산은 GPU 를 사용하도록 내부 분배됨
- 일반적 컴퓨터 아키텍쳐 기반에서, GPU 는 메인보드 BUS(다른 PCI Device 와 대역폭을 공유) 를 통해서만 CPU, RAM 과 통신할 수 있기 때문에 통신폭의 구조적 한계가 있음 
  - 즉 GPU 로의 불필요한 데이터 전송을 최소화 하는 것이 좋음
- 단일 GPU 의 RAM 용량확충은 실질적으로 불가능하므로(즉 비싼 자원임), 현재 런타임 step 에서 GPU 프로세싱에 불필요하다고 보이는 variable 은 최대한 CPU RAM 에 가지고 있는 것이 좋다. 




### 프로그램 리소스의 하드웨어 할당상태 확인 

- device 속성을 통해 실제 variable 이 배치된 위치를 확인
```python
>>> a = tf.Variable(42.0)
>>> a.device
'/job:localhost/replica:0/task:0/device:GPU:0'
>>> b = tf.Variable (42)
>>> b.device
```

- 명시적으로 특정 디바이스에 variable 할당
```python
>>> with tf.device("/cpu:0"):
...     c = tf.Variable(42.0
)
...
>>> c.device
'/job:localhost/replica:0/task:0/device:CPU:0'
```

## 다중 장치에서 병렬 실행

### TF 의 일반적인 그래프 계산 방식
- CPU, GPU 로 DL 그래프를 분할해서 실행할 때,
- CPU 의 evaluation queue 로 이동된 연산은
   - inter-op pool 에서는 모든 CPU thread 공유가 필요한 연산 처리
   - intra-op pool 에서는 inter-op 연산 중 병렬 연산 처리(최대 CPU core 수 만큼) 가능한 연산을 전달 받아 처리
- GPU 의 evaluation queue 로 이동된 연산은
   - 단순하게 순차 처리됨
   - 내부적으로 CUDA, cuDNN 라이브러리에서 멀티스레딩 처리됨
   - GPU hw spec 의 CUDA core 수치로 병렬연산의 척도를 확인할 수 잇음
  

### 예시
- 하기의 A,B,C 는 leaf node
  - A,B 는 inter-op 처리, A 는 병렬처리 가능한 연산이라 가정하면, intra-op 로 전달 및 처리
  - C 는 GPU 에서 바로 처리
- 하위 node 가 처리되면 상위 node 를 처리할 수 있음
  - A,B > F
  - C -> D,E
  - A,B,D,E -> F


![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00212.jpg)

## 분산 전략 샘플 코드

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
def create_model():
    return keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])

In [ ]:
batch_size = 100
model = create_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

distribution = tf.distribute.MirroredStrategy()

# Change the default all-reduce algorithm:
#distribution = tf.distribute.MirroredStrategy(
#    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

# Specify the list of GPUs to use:
#distribution = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# Use the central storage strategy instead:
#distribution = tf.distribute.experimental.CentralStorageStrategy()

#resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.tpu.experimental.initialize_tpu_system(resolver)
#distribution = tf.distribute.experimental.TPUStrategy(resolver)

with distribution.scope():
    model = create_model()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(lr=1e-2),
                  metrics=["accuracy"])

In [ ]:
batch_size = 100 # must be divisible by the number of workers
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

In [ ]:
model.predict(X_new)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

K = keras.backend

distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    model = create_model()
    optimizer = keras.optimizers.SGD()

with distribution.scope():
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().batch(batch_size)
    input_iterator = distribution.make_dataset_iterator(dataset)
    
@tf.function
def train_step():
    def step_fn(inputs):
        X, y = inputs
        with tf.GradientTape() as tape:
            Y_proba = model(X)
            loss = K.sum(keras.losses.sparse_categorical_crossentropy(y, Y_proba)) / batch_size

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return loss

    per_replica_losses = distribution.experimental_run(step_fn, input_iterator)
    mean_loss = distribution.reduce(tf.distribute.ReduceOp.SUM,
                                    per_replica_losses, axis=None)
    return mean_loss

n_epochs = 10
with distribution.scope():
    input_iterator.initialize()
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))
        for iteration in range(len(X_train) // batch_size):
            print("\rLoss: {:.3f}".format(train_step().numpy()), end="")
        print()

In [ ]:
batch_size = 100 # must be divisible by the number of workers
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

# 다중 장치에서 모델 훈련하기


## 모델 병렬화


- DNN : 가로 / 세로 분할 두 경우 다 통신량이 많아서 병렬 성능이 좋지 않음

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00252.jpg)

- CNN : 세로 분할 시 DNN 에 비해 적은 서버간 통신량과 병렬 이득을 볼 수 있음

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00215.jpg)

- RNN : 이론상으로는 각 cell 이 복잡하기 때문에, 가로분할 시 통신량이 많더라도 분할하는게 이득이라지만, in practice 시 1 GPU 가 더 낫다고 함  

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00217.jpg)

## 데이터 병렬화
- 기본적으로 여러 복사된 모델에 데이터를 배치 단위로 분리하여 학습시키는 전략을 말함
- weight 관리 전략으로 크게 Mirrored, Parameterserver 등이 존재 


### Mirrored stretegy
- https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy
- 각각의 모델에서 gradient 를 구한 후, 서버 간 통신으로 **평균값을 취하여, 동일한 update 를 적용 하는 방식**으로 모든 모델들의 상태를 동일하게 유지
- Allreduce 알고리즘으로 효율화

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00218.jpg)


### Parameter server strategy
- https://www.tensorflow.org/api_docs/python/tf/distribute/experimental/ParameterServerStrategy
- 모든 모델의 weight 업데이트를 일치화하는 부분은 동일하지만, weight update 시 동기 / 비동기 방식을 모두 사용할 수 있는 특징이 있음

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00219.jpg)


### 동기 방식
- 모든 분산 모델의 gradient 수집을 기다렷다가 한꺼번에 업데이트 하는 방식
- 결과적으로 Mirrored 와 동일하게 됨
- 어느 한 쪽의 모델 학습 실행이 느려지는 경우 bottleneck 이 될 수 잇음

### 비동기 방식
- 수집단계가 없이, 각각의 모델들에서 gradient 가 계산되는 시점에 즉시 parameter server 의 weight 를 업데이트
- bottleneck 이 없이 머신 성능을 최대한 사용할 수 있는 장점
- 특정 모델에서 실행지연이 일어날시, stale gradient 문제가 발생할 수 있음
  - 특정 gradient 의 늦은 적용으로 전체적인 학습 방향성에 방해를 줄 수 잇음
- stale gradient 문제의 완화법
  - learning rate 감소
  - stale gradient 를 버리거나 크기축소
  - minibatch 크기를 조절
  - 1개 모델만 사용하여 초기 몇번의 epoch 을 돌려(param server update 포함)놓음 (warmup). 학습 초기에 gradient 가 제각각일 확률이 크다고 가정하는 방식으로 보임.
  
### 연구 근황(?)  
- 구글의 최근논문에서는, 일부의 복제모델을 통한 동기화 방식을 사용해보니, 모델 성능과 리소스 효율 두 가지를 합리적으로 끌어올릴 수 있엇다고 함. 

![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00227.jpg)


### 대역폭의 포화
- 다수의 GPU RAM 또는 다수의 서버장비 간에 gradient 를 통신하게 되됨
 - 복제 모델의 수가 임계치만큼 늘어나게 되면, 결국 하드웨어 BUS 또는 네트워크 대역의 한계로 성능의 이점을 잃게 됨 
 - 대체로 작은 모델을 많은 데이터로 학습하는 경우에는, GRAM bandwidth 가 높고 성능이 좋은 GPU 단일 머신을 사용하는 것이 낫다고 한다.
- 대규모의 dense model 성능 문제 해결을 위한 여러 연구가 진행되고 잇음
 - peer-to-peer 파라미터 서버, 손실모델 압축, 복제 모델간 조건부 통신을 통한 최적화 연구 등 

## 분산 전략 API 를 사용한 대규모 훈련
- 추상화된 전략 API 메서드를 제공

```python
distribution = tf.distribute.MirroredStrategy()
with distribution.scope():
    model = create_model()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(lr=1e-2),
                  metrics=["accuracy"])
```

-  분산 전략 세부 설정

```python
# all-reduce algorithm 변경
distribution = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
# GPU 목록 지정
distribution = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
# 중앙 파라미터 전략
distribution = tf.distribute.experimental.CentralStorageStrategy()
# TPU 환경
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.tpu.experimental.initialize_tpu_system(resolver)
distribution = tf.distribute.experimental.TPUStrategy(resolver)
```

## 텐서플로 클러스터에서 모델 훈련


### Tensorflow Cluster
- TF 훈련 또는 실행을 병렬로 실행하고 통신하는 TF 프로세스 그룹
- Cluster 의 각 TF 프로세스는 Task 또는 TF server 로 호칭
 - IP, Port, Type(Role/Job) 속성을 가짐

#### Task 의 Type
- worker :  일반적으로 하나 이상의 GPU가 있는 머신에서 계산을 수행
- chief : 계산 수행 및 TensorBoard 로그 작성,체크 포인트 저장 등의 추가 작업 처리
 - 클러스터에는 단일한 chief 존재, 미지정 시 첫 Task가 chief
- ps(parameter server) : 변수 값만 추적하며 일반적으로 CPU 전용 시스템에 존재
- evaluator : evaluation 처리. 일반적으로 1 클러스터에 1 evaluator 가 존재




#### 클러스터 지정
- 사용할 모든 Task를 선 정의
```json
cluster_spec = {
    "worker" : [
        "machine-a.example.com:2222" ,   # /job:worker/task:0
        "machine-b.example.com:2222"    # /job:worker/task:1
    ],
    "ps" : [ "machine-a.example.com:2221" ] # /job:ps/task:0
}
```
![대체 텍스트](http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/Image00286.jpg)

In [ ]:
import os
import json

os.environ["TF_CONFIG"] = json.dumps({
    "cluster": {
        "worker": ["my-work0.example.com:9876", "my-work1.example.com:9876"],
        "ps": ["my-ps0.example.com:9876"]
    },
    "task": {"type": "worker", "index": 0}
})
print("TF_CONFIG='{}'".format(os.environ["TF_CONFIG"]))

In [ ]:
import tensorflow as tf

resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
worker0 = tf.distribute.Server(resolver.cluster_spec(),
                               job_name=resolver.task_type,
                               task_index=resolver.task_id)

In [ ]:
cluster_spec = tf.train.ClusterSpec({
    "worker": ["127.0.0.1:9901", "127.0.0.1:9902"],
    "ps": ["127.0.0.1:9903"]
})

In [ ]:
#worker1 = tf.distribute.Server(cluster_spec, job_name="worker", task_index=1)
ps0 = tf.distribute.Server(cluster_spec, job_name="ps", task_index=0)

In [ ]:
os.environ["TF_CONFIG"] = json.dumps({
    "cluster": {
        "worker": ["127.0.0.1:9901", "127.0.0.1:9902"],
        "ps": ["127.0.0.1:9903"]
    },
    "task": {"type": "worker", "index": 1}
})
print(repr(os.environ["TF_CONFIG"]))

In [ ]:
distribution = tf.distribute.experimental.MultiWorkerMirroredStrategy()

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

os.environ["TF_CONFIG"] = json.dumps({
    "cluster": {
        "worker": ["127.0.0.1:9901", "127.0.0.1:9902"],
        "ps": ["127.0.0.1:9903"]
    },
    "task": {"type": "worker", "index": 1}
})
#CUDA_VISIBLE_DEVICES=0 

with distribution.scope():
    model = create_model()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(lr=1e-2),
                  metrics=["accuracy"])

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# At the beginning of the program (restart the kernel before running this cell)
distribution = tf.distribute.experimental.MultiWorkerMirroredStrategy()

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis] / 255.
X_test = X_test[..., np.newaxis] / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

n_workers = 2
batch_size = 32 * n_workers
dataset = tf.data.Dataset.from_tensor_slices((X_train[..., np.newaxis], y_train)).repeat().batch(batch_size)
    
def create_model():
    return keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])

with distribution.scope():
    model = create_model()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(lr=1e-2),
                  metrics=["accuracy"])

model.fit(dataset, steps_per_epoch=len(X_train)//batch_size, epochs=10)

In [ ]:
# Hyperparameter tuning

# Only talk to ps server
config_proto = tf.ConfigProto(device_filters=['/job:ps', '/job:worker/task:%d' % tf_config['task']['index']])
config = tf.estimator.RunConfig(session_config=config_proto)
# default since 1.10

In [ ]:
strategy.num_replicas_in_sync

## GCP AI 플랫폼으로 대규모 훈련 실행하기
- gcloud cli shell 를 통해 job 을 trigger 하는 방식
- 소스 코드 및 런타임을 GCP VM 에 설치 하고 실행

```bash
$ gcloud ai-platform jobs submit training my_job_20190531_164700 \
    --region asia-southeast1 \
    --scale-tier PREMIUM_1 \
    --runtime-version 2.0 \
    --python-version 3.5 \
    --package-path /my_project/src/trainer \
    --module-name trainer.task \
    --staging-bucket gs://my-staging-bucket \
    --job-dir gs://my-mnist-model-bucket/trained_model \
    --my-extra-argument1 foo --my-extra-argument2 bar
```

# Reference for media resources
- http://reader.epubee.com/books/mobile/db/db4791ccfa2bab56aa04efa5d23d9336/text00013.html